In [24]:
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import (SystemMessagePromptTemplate, 
HumanMessagePromptTemplate, ChatPromptTemplate)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


In [25]:
load_dotenv(find_dotenv())
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

In [26]:
sentiment_analysis_prompt= """Given a user review below, classify it as a Positive or Negative review, 
Return your answer in only one Classification  word.
review: {review}
Classification: """

sentiment_template = ChatPromptTemplate.from_template(sentiment_analysis_prompt)

sentiment_chain = sentiment_template | llm | StrOutputParser()

sentiment = sentiment_chain.invoke({'review':
                          'I did not like the product'})

In [27]:
sentiment

'Negative'

In [28]:
positive_response = """You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Review: {review}
                Answer:"""

positive_template = ChatPromptTemplate.from_template(positive_response)

positive_chain = positive_template | llm | StrOutputParser()

In [29]:
negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'prashant@company.com'.
                Review: {review}
                Answer:"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)

negative_chain = negative_template | llm | StrOutputParser()

In [30]:
def decisionReviewer(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [31]:
full_chain = {'sentiment': sentiment_chain, 'review': lambda x : x['review']} | RunnableLambda(decisionReviewer)

full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template='Given a user review below, classify it as a Positive or Negative review, \nReturn your answer in only one Classification  word.\nreview: {review}\nClassification: '), additional_kwargs={})])
             | ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x00000156966CEFD0>, async_client=<openai.resources.chat.completions.

In [32]:
review = 'I liked the product'

final_output = full_chain.invoke({'review':review})

final_output

"Thank you so much for your positive feedback! We're thrilled to hear that you liked the product. If you have a moment, we would love for you to share your experience on social media. Your support helps us grow and reach more amazing customers like you! Thank you again for choosing us!"

Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda

In [37]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

def pricing(text):
    return len(text.split()) *0.1

text_input = ChatPromptTemplate.from_template('Explaing this topic: {input1} in {length} sentences')

text_input

ChatPromptTemplate(input_variables=['input1', 'length'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input1', 'length'], input_types={}, partial_variables={}, template='Explaing this topic: {input1} in {length} sentences'), additional_kwargs={})])

In [36]:
chain = text_input | llm | StrOutputParser()

output = chain.invoke({
    'input1':'Finance',
    'length': 2
})

output

'Finance is the study and management of money, investments, and other financial instruments, focusing on how individuals, businesses, and governments acquire, allocate, and use resources over time. It encompasses various areas, including personal finance, corporate finance, and public finance, each addressing different aspects of financial decision-making and risk management.'

In [39]:
chain = text_input | llm | StrOutputParser() | {'char_counts': RunnableLambda(char_counts), 
                                            'word_counts': RunnableLambda(word_counts), 
                                            'cost': RunnableLambda(pricing),
                                            'output': RunnablePassthrough()}

output = chain.invoke({
    'input1':'Finance',
    'length': 2
})

output


{'char_counts': 275,
 'word_counts': 36,
 'cost': 3.6,
 'output': 'Finance is the management of money and investments, encompassing activities such as borrowing, lending, budgeting, saving, and investing. It involves analyzing and strategizing to maximize the value of financial resources, whether for individuals, businesses, or governments.'}

Custom chain usin @Chian Decorator:

In [41]:
# One chain with System and Human templates:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')


messages = [system, question]
template = ChatPromptTemplate(messages)

fact_chain = template | llm | StrOutputParser()

output = fact_chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 2})

output

'1. The solar system consists of the Sun and eight planets, including Earth.  \n2. It also contains moons, asteroids, comets, and dwarf planets like Pluto.'

In [ ]:
# another chain with reusing system prompt:

question = HumanMessagePromptTemplate.from_template('write a poem in {topics}, in {sentences} lines')

message = [system, question]
template2 = ChatPromptTemplate(message)
poem_chain = template2 | llm | StrOutputParser()

output = poem_chain.invoke({'school': 'primary', 'topics': 'solar system', 'sentences': 2})

output

'In the vast sky, planets dance and twirl,  \nStars shine bright, in a cosmic swirl.'

In [46]:
from langchain_core.runnables import chain

@chain
def custom_chains(params):
    return {
        'fact': fact_chain.invoke(params),
        'poem': poem_chain.invoke(params)
    }

params = {'school': 'primary', 'topics': 'romance', 'points': 2, 'sentences': 3}

output = custom_chains.invoke(params)

print(output['fact'])

print('\n')

print(output['poem'])

1. Romance often involves feelings of love and attraction between people.  
2. It can include gestures like dating, gifts, and expressing emotions.


In twilight's glow, your eyes ignite,  
Whispers of love dance through the night,  
Hearts entwined, forever in flight.
